In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE


In [3]:
df = pd.read_csv("/content/Creditcard_data.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
print(df['Class'].value_counts)

<bound method IndexOpsMixin.value_counts of 0      0
1      1
2      0
3      0
4      0
      ..
767    0
768    0
769    0
770    0
771    0
Name: Class, Length: 772, dtype: int64>


In [5]:
X = df.drop("Class", axis = 1)
y = df["Class"]

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
smote = SMOTE(random_state = 42)
X_bal , y_bal = smote.fit_resample(X_scaled, y)

In [14]:
balanced_df = pd.DataFrame(X_bal)
balanced_df['Class']=y_bal

In [15]:
print(balanced_df['Class'].value_counts())

Class
0    763
1    763
Name: count, dtype: int64


In [23]:
def simple_random_sampling(df, n):
  return df.sample(n=n, random_state = 42)
def systematic_sampling(df, k=5):
  return df.iloc[::k]
def stratified_sampling(df,n):

  from sklearn.model_selection import StratifiedShuffleSplit
  split = StratifiedShuffleSplit(n_splits=1, train_size=n, random_state=42)
  for train_idx , _ in split.split(df, df['Class']):

    return df.iloc[train_idx]
def cluster_sampling(df, n_clusters = 5):
  df_copy = df.copy()
  df_copy['cluster'] = pd.qcut(df_copy.index, n_clusters, labels = False)
  chosen_cluster = np.random.choice(df_copy['cluster'].unique())
  return df_copy[df_copy['cluster']== chosen_cluster].drop("cluster",axis = 1)
def bootstrap_sampling(df,n):
  return df.sample(n=n, replace = True, random_state = 42)


In [24]:
sample_size = 1000
S1 = simple_random_sampling(balanced_df, sample_size)
S2 = systematic_sampling(balanced_df)
S3 = stratified_sampling(balanced_df, sample_size)
S4 = cluster_sampling(balanced_df)
S5 = bootstrap_sampling(balanced_df, sample_size)
samples = [S1, S2, S3, S4, S5]

In [25]:
models = {
    "m1_logistic": LogisticRegression(max_iter = 500),
    "M2_DecisionTree": DecisionTreeClassifier(),
    "M3_RandomForest": RandomForestClassifier(),
    "M4_KNN": KNeighborsClassifier(),
    "M5_SVM": SVC()
}

In [30]:
results = pd.DataFrame(index = models.keys(),
                       columns = ["Sampling1", "Sampling2", "Sampling3", "Sampling4", "Sampling5"])
for i , sample in enumerate(samples):
  X_s = sample.drop("Class", axis=1)
  y_s = sample["Class"]

  for model_name , model in models.items():
    scores = cross_val_score(model, X_s, y_s, cv = 5, scoring = "accuracy")
    results.iloc[list(models.keys()).index(model_name),i]=round(scores.mean()*100,2)

In [31]:
print(results)

                Sampling1 Sampling2 Sampling3 Sampling4 Sampling5
m1_logistic          91.2      85.3      92.2     93.11      94.2
M2_DecisionTree      97.4     87.24      98.2     81.97      98.1
M3_RandomForest      99.7     98.69      99.8     96.72      99.4
M4_KNN               92.4     78.11      92.6     90.49      92.9
M5_SVM               97.7     95.09      97.8     98.03      98.8


In [32]:
print(results.astype(float).idxmax(axis=1))

m1_logistic        Sampling5
M2_DecisionTree    Sampling3
M3_RandomForest    Sampling3
M4_KNN             Sampling5
M5_SVM             Sampling5
dtype: object
